In [35]:
import numpy as np
import pandas as pd

In [36]:
caInputeDF = pd.read_csv("ConJointInput.csv", sep = ";")

Input = a ranking of 3 different features (TV Size, TV Type, TV Color) with 3(32",37",42")<br>
, 2 (LCD, Plasma), 3(Black, Silver, Anthrazit) different stimuli types.

In [37]:
caInputeDF

,Stimulus,Rank
0,A1B1C1,2
1,A1B1C2,3
2,A1B1C3,1
3,A1B2C1,5
4,A1B2C2,6
5,A1B2C3,4
6,A2B1C1,8
7,A2B1C2,9
8,A2B1C3,7
9,A2B2C1,11


## First step is to introduce dummy variables for every stimulus<p>
There are in total 9 differen stimuli, and 18 different combinations


In [38]:
ConjointDummyDF = pd.DataFrame(np.zeros((18,9)), columns=["Rank","A1", "A2", "A3",
                                                    "B1","B2", 
                                                    "C1", "C2",
                                                    "C3"])

In [39]:
ConjointDummyDF.Rank = caInputeDF.Rank

for index, row in caInputeDF.iterrows(): 
    stimuli1, stimuli2, stimuli3 = caInputeDF["Stimulus"][index][:2], \
    caInputeDF["Stimulus"][index][2:4], caInputeDF["Stimulus"][index][4:6]
    
    
    ConjointDummyDF.loc[index, [stimuli1,stimuli2,stimuli3]] = 1

In [40]:
ConjointDummyDF

,Rank,A1,A2,A3,B1,B2,C1,C2,C3
0,2,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,5,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,6,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
5,4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
6,8,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
7,9,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
8,7,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
9,11,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0


In [41]:
ConjointDummyDF.head()

,Rank,A1,A2,A3,B1,B2,C1,C2,C3
0,2,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,5,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,6,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


## Insert the proper Stimulus names

In [42]:
fullNames = {"Rank":"Rank", \
           "A1": "32\" (81cm)","A2": "37\" (94cm)","A3": "42\" (107cm)", \
          "B1": "Plasma", "B2":"LCD", \
           "C1":"Silver", "C2":"Black", "C3": "Anthrazit",\
          }

ConjointDummyDF.rename(columns=fullNames, inplace=True)

In [43]:
ConjointDummyDF.head()

,Rank,"32"" (81cm)","37"" (94cm)","42"" (107cm)",Plasma,LCD,Silver,Black,Anthrazit
0,2,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,5,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,6,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


## Estimate Main Effects with a linear regression
<p>

There are different ways for parameter estimation beside linear regression depending on what kind of rating you have.<br>
For example using Probit or Logit is the output is not a rank but a decision (1=chose stimulus, 0 = no choice).

In [44]:
import statsmodels.api as sm

In [45]:
ConjointDummyDF.columns

Index(['Rank', '32" (81cm)', '37" (94cm)', '42" (107cm)', 'Plasma', 'LCD',
       'Silver', 'Black', 'Anthrazit'],
      dtype='object')

In [50]:
X = ConjointDummyDF[[u'32" (81cm)', u'37" (94cm)', u'42" (107cm)', u'Plasma',\
       u'LCD', u'Silver', u'Black', u'Anthrazit']]
X = sm.add_constant(X) 
Y = ConjointDummyDF.Rank
linearRegression = sm.OLS(Y, X). fit()
linearRegression.summary()

/home/xenakas/.local/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1477: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   Rank   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 5.582e+30
Date:                Mon, 06 Mar 2023   Prob (F-statistic):          9.26e-182
Time:                        15:24:27   Log-Likelihood:                 574.12
No. Observations:                  18   AIC:                            -1136.
Df Residuals:                      12   BIC:                            -1131.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           4.3846   4.53e-16   9.67e+15      0.000       4.385       4.385
32" (81cm)     -4.5385    1.4e-15  -3.25e+15      0.000      -4.538      -4.538
37" (94cm)      1.4615    1.4e-15   1.05e+15      0.000       1.462       1.462
42" (107cm)     7.4615    1.4e-15   5.34e+15      0.000       7.462       7.462
Plasma          0.6923   1.01e-15   6.87e+14      0.000       0.692       0.692
LCD             3.6923   1.01e-15   3.66e+15      0.000       3.692       3.692
Silver          1.4615    1.4e-15   1.05e+15      0.000       1.462       1.462
Black           2.4615    1.4e-15   1.76e+15      0.000       2.462       2.462
Anthrazit       0.4615    1.4e-15    3.3e+14      0.000       0.462       0.462
==============================================================================
Omnibus:                        0.302   Durbin-Watson:                   0.894
Prob(Omnibus):                  0.860   Jarque-Bera (JB):                0.464
Skew:                           0.084   Prob(JB):                        0.793
Kurtosis:                       2.232   Cond. No.                     1.78e+16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.23e-31. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [47]:
X = ConjointDummyDF[[u'32" (81cm)', u'37" (94cm)',u'Plasma',\
     u'Silver', u'Black']]
X = sm.add_constant(X)
Y = ConjointDummyDF.Rank
linearRegression = sm.OLS(Y, X). fit()
linearRegression.summary()

/home/xenakas/.local/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1477: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   Rank   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 2.678e+30
Date:                Mon, 06 Mar 2023   Prob (F-statistic):          7.60e-180
Time:                        15:22:43   Log-Likelihood:                 567.51
No. Observations:                  18   AIC:                            -1123.
Df Residuals:                      12   BIC:                            -1118.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         16.0000   3.47e-15   4.61e+15      0.000      16.000      16.000
32" (81cm)   -12.0000   3.47e-15  -3.46e+15      0.000     -12.000     -12.000
37" (94cm)    -6.0000   3.47e-15  -1.73e+15      0.000      -6.000      -6.000
Plasma        -3.0000   2.84e-15  -1.06e+15      0.000      -3.000      -3.000
Silver         1.0000   3.47e-15   2.88e+14      0.000       1.000       1.000
Black          2.0000   3.47e-15   5.76e+14      0.000       2.000       2.000
==============================================================================
Omnibus:                        1.432   Durbin-Watson:                   0.812
Prob(Omnibus):                  0.489   Jarque-Bera (JB):                0.954
Skew:                           0.232   Prob(JB):                        0.621
Kurtosis:                       1.972   Cond. No.                         4.94
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Part worth values & relative importance of the stimuli
<p>
Importance of Stimuli= Max(beta) - Min(beta)
<br>
Relative Importance of Stimuli = Importance of Stim / Sum(Importance of all Stimuli)

In [51]:
importance = []
relative_importance = []

rangePerFeature = []

begin = "A"
tempRange = []
for stimuli in fullNames.keys():
    if stimuli[0] == begin:
        tempRange.append(linearRegression.params[fullNames[stimuli]])
    elif stimuli == "Rank":
        rangePerFeature.append(tempRange)
    else:
        rangePerFeature.append(tempRange)
        begin = stimuli[0]
        tempRange = [linearRegression.params[fullNames[stimuli]]]
        

In [52]:
for item in rangePerFeature:
    importance.append( max(item) - min(item))

for item in importance:
    relative_importance.append(100* round(item/sum(importance),3))

In [53]:

partworths = []

item_levels = [1,3,5,8]

for i in range(1,4):
    part_worth_range = linearRegression.params[item_levels[i-1]:item_levels[i]]
    print(part_worth_range)

32" (81cm)   -4.538462
37" (94cm)    1.461538
dtype: float64
42" (107cm)    7.461538
Plasma         0.692308
dtype: float64
LCD       3.692308
Silver    1.461538
Black     2.461538
dtype: float64


In [54]:
meanRank = []
for i in ConjointDummyDF.columns[1:]:
    newmeanRank = ConjointDummyDF["Rank"].loc[ConjointDummyDF[i] == 1].mean()
    meanRank.append(newmeanRank)

    
#total Mean or, "basic utility" is used as the "zero alternative"
totalMeanRank = sum(meanRank) / len(meanRank)



partWorths = {}
for i in range(len(meanRank)):
    name = fullNames[sorted(fullNames.keys())[i]]
    partWorths[name] = meanRank[i] - totalMeanRank

In [55]:
partWorths

{'32" (81cm)': -6.0,
 '37" (94cm)': 0.0,
 '42" (107cm)': 6.0,
 'Plasma': -1.5,
 'LCD': 1.5,
 'Silver': 0.0,
 'Black': 1.0,
 'Anthrazit': -1.0}

### Summary & Results

In [56]:
print("Relative Importance of Feature:\n\nMonitor Size:",relative_importance[0], "%",\
"\nType of Monitor:", relative_importance[1], "%", "\nColor of TV:", relative_importance[2], "%\n\n")

print("--"*30)

print("Importance of Feature:\n\nMonitor Size:",importance[0],\
"\nType of Monitor:", importance[1],  "\nColor of TV:", importance[2])

Relative Importance of Feature:

Monitor Size: 44.4 % 
Type of Monitor: 44.4 % 
Color of TV: 11.1 %


------------------------------------------------------------
Importance of Feature:

Monitor Size: 12.000000000000002 
Type of Monitor: 12.000000000000002 
Color of TV: 3.0000000000000053


What would be the optimal product bundle? <p>
42", LCD, Black

In [58]:
#As array that looks like X
#Must include Constant!

optBundle = [1,0,0,1,0,1,0,1,0]
print("The best possible Combination of Stimuli would have the highest rank:",\
linearRegression.predict(optBundle)[0])

The best possible Combination of Stimuli would have the highest rank: 18.0


Or using the Partworths:

In [59]:
#Optimal Bundle:
#42", LCD, Black

optimalWorth = partWorths["42\" (107cm)"] + partWorths["LCD"] + partWorths["Black"]

print("Choosing the optimal Combination brings the user an additional ", optimalWorth, "'units' of utility")

Choosing the optimal Combination brings the user an additional  8.5 'units' of utility
